[Here are the slides](https://docs.google.com/presentation/d/11uyJMpCyVceYlC50QUdVp3DJe42BLxW4CHykxIdR7AU/edit?usp=sharing)


# Opening Images in Python

In [ ]:
from PIL import Image

img = Image.open("alice.png")
img.show()

# Asking User to Verify a PNG Signature 

Requesting user input. Does the supplied signature photo make sense?

In [ ]:
def validate():
    """Just validate one transfer"""
    user_input = input("Is this a valid signature? (y/n)")
    if user_input.lower() == "y":
        print("It's valid!")
    elif user_input.lower() == "n":
        print("It's not valid!")
    else:
        validate()

In [ ]:
def display_and_verify(filename):
    img = Image.open(filename)
    img.show()
    validate()

In [ ]:
display_and_verify("alice.png")

# Defining the Coin

Stupid-simple Python class

In [ ]:
class PNGCoin:
    def __init__(self, transfers):
        self.transfers = transfers  # Instances of PIL.Image

# Validating a Coin

Here we check every entry in PNGCoin.transfers

In [ ]:
def handle_user_input(user_input):
    if user_input.lower() == "y":
        return True
    elif user_input.lower() == "n":
        return False
    else:
        user_input  = input('Please enter "y" or "n"')
        return handle_user_input(user_input)

def validate(coin):
    for transfer in coin.transfers:
        transfer.show()
        user_input = input("Is this a valid signature? (y/n)")
        valid = handle_user_input(user_input)
        if not valid:
            return False
    return True

In [ ]:
coin = PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob.png")
])
validate(coin)

In [ ]:
# A forgery
coin = PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob-forged.png")
])
validate(coin)


# Serializing Coins 

Let's take the coin defined ^^ and write it to disk with help from `pickle`

In [ ]:
import pickle

def serialize(coin):
    return pickle.dumps(coin)

In [ ]:
def to_disk(coin, filename):
    serialized = serialize(coin)
    with open(filename, "wb") as f:
        f.write(serialized)

In [ ]:
import os

def file_exists(filename):
    return os.path.isfile(filename)


In [ ]:
file_exists("coin.pngcoin")

In [ ]:
to_disk(coin, "coin.pngcoin")

In [ ]:
file_exists("coin.pngcoin")

# Deserializing Coins

Let's take the `coin.pngcoin` file we created ^^ and read it back into Python with help from `pickle`

In [ ]:
def deserialize(serialized):
    return pickle.loads(serialized)

In [ ]:
coin2 = deserialize(serialize(coin))

print(coin2.transfers == coin.transfers)

In [ ]:
def from_disk(filename):
    with open(filename, "rb") as f:
        serialized = f.read()
        return deserialize(serialized)

In [ ]:
coin3 = from_disk("coin.pngcoin")

print(coin3.transfers == coin.transfers)

# Using the Final Library

I also wrote a [pngcoin.py](pngcoin.py) library that uses a more object-oriented design. Here's how it works:

In [ ]:
import pngcoin

In [ ]:
coin = pngcoin.PNGCoin([
    Image.open("alice.png"),
    Image.open("alice-to-bob.png")
])

In [ ]:
coin.validate()

In [ ]:
coin.to_disk("library-example.pngcoin")

In [ ]:
coin = pngcoin.PNGCoin.from_disk("library-example.pngcoin")
coin.validate()